In [1]:
from qiskit import *
import numpy as np
#[j=0]     H2molecule: generate Eigenstates E0,E1,E2,E3
n_site=2  
N=n_site 
j_coupling=1
a3=-0.0115
a0=0
a00=-1.04391
a2=-0.42045
J=0.42045 
a1=J
a4=0.179005

step_max=288*3
t_max=36
dt=t_max/step_max


### simulator and Shots
simulator=Aer.get_backend('qasm_simulator')
n_shots=10000000#10^7

##### define quantum circuit
qr=QuantumRegister(14,'qr')
cr=ClassicalRegister(14,'cr')
qc=QuantumCircuit(qr, cr)

### measurement
qc_meas=QuantumCircuit(qr,cr)
qc_meas.measure(qr,cr)

### LIst
time_list_qiskit=[]
mag_list_qiskit=[]
magx_list_qiskit=[]
magy_list_qiskit=[]

#####  Important : Initial state
### E3     qc.x(qr[12]) 
### E2     qc.x(qr[13]),qc.x(qr[12]), qc.h(qr[13]),qc.h(qr[12])
### E1     qc.x(qr[13]), qc.h(qr[13]),qc.h(qr[12])
### E0     qc.x(qr[13])

qc.x(qr[13]) 
#qc.x(qr[12])     

#qc.h(qr[13])
#qc.h(qr[12])

#################################   Z:measurement
  
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

############################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                           
        obs=obs+1
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/n_shots
        else:
            mag00=mag00

print('mag00=',mag00)

    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))  
        if j>=n_site-2:                    
            mag=mag+a1*obs*count/n_shots/(1)
        else:
            mag=mag

mag_list_qiskit.append(mag)
print('mag=',mag)
#################################
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))  
        if j>=n_site-2:                    
            mag2=mag2+a2*obs*count/n_shots/(1)
        else:
            mag2=mag2
#time_list_qiskit.append(dt*step)
#mag_list_qiskit.append(mag)
print('mag2=',mag2)

############################################  measurement:XX-energy
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*count
      
    magx=a4*obs/n_shots
print('magx=', magx)

#################################
    
qc_total_zz=qc.compose(qc_meas)
  

job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*count
      
    magzz=a3*obs/n_shots
print('magzz=', magzz)

##################################################
print('EE=',mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)




mag00= 0.0
mag= -0.42045
mag2= -0.42045
magx= -8.928769399999999e-05
magzz= 0.0115
EE= -0.8294892876939999
E0= -1.8733992876939998


In [2]:
# [j=1]   ### Second-order Suzuki-Trotter formula :  the first group of the twirling operations
##################################  
import numpy as np
E=-0.829489 ###  substitute EE in [j=0]
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[10])
qc.h(qr[11])

#################################################
qc.s(qr[10])


for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[10]) 
### JZ0                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[10],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[10],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])

### Z1                                                
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[10],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[10],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[10],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[10],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[10],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13])    

### ZZ
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    

### Z1                                                
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[10],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[10],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[10],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[10],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
########################################################################### 
for stepBB in range (2):     
    qc.s(qr[11])
    
    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[11]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[11],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[11],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        
### Z1                                                
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])   
        qc.cx(qr[11],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

#### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[11],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[11],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[11],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[11],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 

### ZZ  
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
        
### Z1                                                
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[11],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[11],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[11],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[11],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
##########################################################################           

#################################################################
    
qc.h(qr[10])
qc.h(qr[11])

######################################### 
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)


zero=0
for bit_string, count in counts.items():
    zero=zero+(1-int(bit_string[2]))*(1-int(bit_string[3]))*count 
print('zero=', zero)

###########################################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                            
        obs=obs+1*(1-int(bit_string[2]))*(1-int(bit_string[3]))  
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
############################################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))  
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag
mag_list_qiskit.append(mag)
print('mag=',mag)
#######################################
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*(1-int(bit_string[2]))*(1-int(bit_string[3]))  
        if j>=n_site-2:                                     
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2
#time_list_qiskit.append(dt*step)
mag_list_qiskit.append(mag)
print('mag2=',mag2)

############################################  XX-energy
    
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
  #  qc_total=qc+qc_meas  OLD
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
#print('xx-counts=', counts)
       
magx=0
obs=0
obsy=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[j]))+(int(bit_string[j+1]))))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*count
        obsy=obsy+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*count
        
    magx=a4*obs/(zero)/(1)
    magy=a4*obsy/(zero)/(1)
       
print('magx=', magx)
print('magy=', magy)
##############################################
magxx=0
obsxx=0
for bit_string, count in counts.items():
    for j in range(1):
        obsxx=obsxx+((-1)**((int(bit_string[j]))+(int(bit_string[j+1]))))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*count
                                          
    magxx=a4*obsxx/(zero)/(1)
       
print('magxx=', magxx)
#################################
    
qc_total_zz=qc.compose(qc_meas)
    

job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
#print('xx-counts=', counts)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[j]))+(int(bit_string[j+1]))))*(1-int(bit_string[2]))*(1-int(bit_string[3]))*count
       
    magzz=a3*obs/(zero)

print('magzz=', magzz)


print('EE=',mag00+ magzz+mag+mag2+magx)
print('E0=',a00+magzz+mag+mag2+magx)






zero= 9875491
mag00= 0.0
mag= -0.4113108294007862
mag2= -0.4113108294007862
magx= -0.03694590074609961
magy= -0.03694590074609961
magxx= -0.03694590074609961
magzz= 0.011499055591261234
EE= -0.8480685039564108
E0= -1.8919785039564108


In [3]:
###[j=2]         the second group of the twirling operations
from functools import reduce
from operator import mul

E=-0.848069 ###  substitute EE in [j=1]
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[8])
qc.h(qr[9])

#################################################
qc.s(qr[8])


for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[8]) 
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[8],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[8],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])

### Z1                                                
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[8],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 

    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[8],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[8],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 

### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[8],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[8],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    
### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    

### Z1                                                
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[8],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[8],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
    
### JZ                                               
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[8],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[8],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
########################################################################### 
for stepBB in range (2):    
    qc.s(qr[9])
    

    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[9]) 
### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[9],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[9],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

### Z1                                              
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[9],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        

### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[9],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[9],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    
    
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[9],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[9],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        

### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[9],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[9],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 


### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[9],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[9],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
##########################################################################           

    
qc.h(qr[8])
qc.h(qr[9])

######################################### 
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

zero=0
for bit_string, count in counts.items():
    zero=zero+(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count 
print('zero=', zero)
   
##########################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                                 
        obs=obs+1*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
######################################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,5)))))     
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag

print('mag=',mag)


######################################################
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,5)))))    
        if j>=n_site-2:                                     
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2

print('mag2=',mag2)

############################################  XX-energy
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
  
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)

       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[j]))+(int(bit_string[j+1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,5)))))*count #*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
       
        magx=a4*obs/(zero)
magx_list_qiskit.append(magx)
print('magx=', magx)

#################################
    

qc_total_zz=qc.compose(qc_meas)

job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[j]))+(int(bit_string[j+1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,5)))))*count 
       
    magzz=a3*obs/(zero)
print('magzz=', magzz)

##########################################
print('EE=', mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)




zero= 9874431
mag00= 0.0
mag= -0.41120838097911666
mag2= -0.41120838097911666
magx= -0.037280940908898955
magzz= 0.011501007399818785
EE= -0.8481966954673135
E0= -1.8921066954673136


In [4]:
#[j=3]    the third group of the twirling operations

E=-0.8481967 ###  substitute EE in [j=2]
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[6])
qc.h(qr[7])

#################################################
qc.s(qr[6])


for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[6]) 
### JZ                                              
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[6],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[6],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[6],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ

    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12])
    
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[6],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[6],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    
    
#### YY nashi

### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[6],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[6],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    

### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    
### Z1                                           
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[6],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[6],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
    
### JZ                                               
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[6],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[6],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
#################################################
for stepBB in range (2): 
    qc.s(qr[7])
     

    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[7])
### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[7],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[7],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

### Z1                                              
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[7],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[7],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[7],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 

### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[7],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[7],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### ZZ
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12])
        
### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[7],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[7],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[7],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[7],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

#################################################################
    
qc.h(qr[6])
qc.h(qr[7])

######################################### 2023-1-2
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

zero=0
for bit_string, count in counts.items():
    zero=zero+((reduce(mul,((1-int(bit_string[i])) for i in range(2,7))))) *count 
print('zero=', zero)

###########################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                                 
        obs=obs+1*((reduce(mul,((1-int(bit_string[i])) for i in range(2,7)))))
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
#############################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,7)))))    
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag
#time_list_qiskit.append(dt*step)
mag_list_qiskit.append(mag)
print('mag=',mag)
#print(bit_string[0])
#print(bit_string[1])
#print(bit_string[2])
#print(bit_string[3])

#############################################
    
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,7)))))    
        if j>=n_site-2:                                     
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2
print('mag2=',mag2)

############################################  
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
#print('xx-counts=', counts)
       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,7)))))*count 
    magx=a4*obs/(zero)

print('magx=', magx)

#################################
    
qc_total_zz=qc.compose(qc_meas)
  
    

job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,7)))))*count 
    magzz=a3*obs/(zero)
print('magzz=', magzz)

########################################
print('EE=', mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)







zero= 9874941
mag00= 0.0
mag= -0.4112329145409578
mag2= -0.4112329145409578
magx= -0.03743019632421095
magzz= 0.011500341217228538
EE= -0.848395684188898
E0= -1.8923056841888977


In [5]:
#[j=4]       the fourth group of the twirling operations

E=-0.8488396 # substutute EE in j=3
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[4])
qc.h(qr[5])

#################################################
qc.s(qr[4])

for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[4]) 
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[4],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[4],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[4],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ

    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12])
    
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[4],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[4],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 

### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[4],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[4],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    

### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    
### Z1                                              
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[4],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[4],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
       
### JZ                                               
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[4],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[4],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
###################################################
for stepBB in range (2): 
    qc.s(qr[5])
     

    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[5])
### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[5],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[5],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[5],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[5],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[5],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[5],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### ZZ
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12])
        
### Z1                                              
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[5],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[5],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[5],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[5],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
######################################################
    
qc.h(qr[4])
qc.h(qr[5])

######################################### 
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

zero=0
for bit_string, count in counts.items():
    zero=zero+((reduce(mul,((1-int(bit_string[i])) for i in range(2,9))))) *count 
print('zero=', zero)

###########################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                           
        obs=obs+1*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
#############################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))    
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag
mag_list_qiskit.append(mag)
print('mag(ZI)=',mag)

#############################################
    
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))    
        if j>=n_site-2:                                   
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2
print('mag2(IZ)=',mag2)

############################################  XX-energy
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
  
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))*count #*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
    magx=a4*obs/(zero)
print('magx(XX)=', magx)

#################################
#### ZZ  
qc_total_zz=qc.compose(qc_meas)
 
job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))*count 
    magzz=a3*obs/(zero)
print('magzz(ZZ)=', magzz)

###########################################
###XZ
qc.h(qr[13])
qc_total_xz=qc.compose(qc_meas)
qc.h(qr[13])
    

job=execute(qc_total_xz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xz)
       
magxz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))*count #*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
    magxz=1*obs/(zero)
print('XZ=', magxz)

###########################################
###ZX
qc.h(qr[12])
qc_total_zx=qc.compose(qc_meas)
qc.h(qr[12])
    

job=execute(qc_total_zx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zx)
       
magzx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,9)))))*count #*(1-int(bit_string[2]))*(1-int(bit_string[3]))*(1-int(bit_string[4]))*(1-int(bit_string[5]))*count
    magzx=1*obs/(zero)
print('ZX=', magzx)


########################################
print('EE=', mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)

print('ZI=', mag/a1)
print('IZ=', mag2/a2)
print('XX=', magx/a4)
print('ZZ=',magzz/a3)

###########   E0,E3 for tomography




zero= 9874679
mag00= 0.0
mag(ZI)= -0.41129611210146677
mag2(IZ)= -0.41129611210146677
magx(XX)= -0.03733304619319777
magzz(ZZ)= 0.011499707686700498
XZ= -0.0001161556745287619
ZX= 9.55980442503498e-05
EE= -0.8484255627094308
E0= -1.892335562709431
ZI= -0.9782283555749002
IZ= 0.9782283555749002
XX= -0.2085586782112107
ZZ= -0.9999745814522173


In [6]:
#[j=5]   the fifth group of the twirling operations

E=-0.84842556  # substitute EE in j=4
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[2])
qc.h(qr[3])

#################################################
qc.s(qr[2])


for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[2]) 
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[2],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[2],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[2],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ

    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12])
    
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[2],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[2],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[2],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[2],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    

### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[2],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[2],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
    
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[2],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[2],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
########################################################################### 
for stepBB in range (2): 
    qc.s(qr[3])
     

    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[3])
### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[3],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[3],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[3],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[3],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[3],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[3],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[3],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### ZZ
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12])
        
### Z1                                             
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[3],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[3],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[3],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[3],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
#################################################
    
qc.h(qr[2])
qc.h(qr[3])

#####################################
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

zero=0
for bit_string, count in counts.items():
    zero=zero+((reduce(mul,((1-int(bit_string[i])) for i in range(2,11))))) *count 
print('zero=', zero)

###########################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                           
        obs=obs+1*((reduce(mul,((1-int(bit_string[i])) for i in range(2,11)))))
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
#############################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,11)))))   
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag
print('mag=',mag)

#############################################
    
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,11)))))    
        if j>=n_site-2:                                     
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2
print('mag2=',mag2)

############################################  XX-energy
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,11)))))*count 
    magx=a4*obs/(zero)
print('magx=', magx)

#################################
    
qc_total_zz=qc.compose(qc_meas)
    
job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,11)))))*count 
    magzz=a3*obs/(zero)
print('magzz=', magzz)

####################################
print('EE=', mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)



zero= 9875159
mag00= 0.0
mag= -0.4112071463305046
mag2= -0.4112071463305046
magx= -0.037274893468550736
magzz= 0.011500427385523615
EE= -0.8481887587440363
E0= -1.8920987587440363


In [7]:
#[j=6]   the sixth group of the twirling operations

E=-0.848189 # substitute EE in j=5
ss=0.5*np.pi/(E)
Dt=0.01

qc.h(qr[0])
qc.h(qr[1])

#################################################
qc.s(qr[0])


for step1A in range(100):
    qc.p(-1*a0*ss*Dt,qr[0]) 
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[0],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[0],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[0],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
### ZZ

    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12])
    
    
### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[0],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[0],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    

### XX
    qc.cx(qr[12],qr[13])    
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[0],qr[12])
    qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
    qc.cz(qr[0],qr[12])
    qc.rx((0.25*ss)*a4*Dt,qr[12]) 
    qc.cx(qr[12],qr[13]) 
    

### ZZ
    
    qc.cx(qr[13],qr[12])    
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((0.25*ss)*a3*Dt,qr[12]) 
    qc.cx(qr[13],qr[12]) 
    
### Z1                                               
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
    qc.cx(qr[0],qr[12])
    qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
    qc.cx(qr[0],qr[12])
    qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
     
    
### JZ                                                
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13])  
    qc.cx(qr[0],qr[13])
    qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
    qc.cx(qr[0],qr[13])
    qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
#############################################
for stepBB in range (2): 
    qc.s(qr[1])
     

    for step1B in range(100):
        qc.p(-1*a0*ss*Dt,qr[1])
### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[1],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[1],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 

### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12])  
        qc.cx(qr[1],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### ZZ
        
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12]) 
        
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[1],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[1],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    
### XX
        qc.cx(qr[12],qr[13])    
        qc.rx((0.25*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[1],qr[12])
        qc.rx((-0.5*ss)*a4*Dt,qr[12]) 
        qc.cz(qr[1],qr[12])
        qc.rx((0.25*ss)*a4*Dt,qr[12])
        qc.cx(qr[12],qr[13]) 
    

### ZZ
        qc.cx(qr[13],qr[12])    
        qc.rz((0.25*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((-0.5*ss)*a3*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((0.25*ss)*a3*Dt,qr[12])
        qc.cx(qr[13],qr[12])
        
### Z1                                               
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((-0.5*a2*ss)*(step_max/step_max)*Dt,qr[12]) 
        qc.cx(qr[1],qr[12])
        qc.rz((0.25*a2*ss)*(step_max/step_max)*Dt,qr[12]) 

### JZ  
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[1],qr[13])
        qc.rz((-0.5*J*ss)*(step_max/step_max)*Dt,qr[13]) 
        qc.cx(qr[1],qr[13])
        qc.rz((0.25*J*ss)*(step_max/step_max)*Dt,qr[13]) 
###################################################
    
qc.h(qr[0])
qc.h(qr[1])

######################################
qc_total=qc.compose(qc_meas)

job=execute(qc_total, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total)

zero=0
for bit_string, count in counts.items():
    zero=zero+((reduce(mul,((1-int(bit_string[i])) for i in range(2,13))))) *count 
print('zero=', zero)
   
###########################################
mag00=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                       
        obs=obs+1*((reduce(mul,((1-int(bit_string[i])) for i in range(2,13)))))
        if j>=n_site-2:                                     
            mag00=mag00+a0*obs*count/(zero)
        else:
            mag00=mag00

print('mag00=',mag00)
#######################################
    
mag=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,13)))))    
        if j>=n_site-2:                                     
            mag=mag+a1*obs*count/(zero)/(1)
        else:
            mag=mag
print('mag=',mag)

#############################################
    
mag2=0
for bit_string, count in counts.items():
    obs=0
    for j in range(n_site-1):                               
        obs=obs+(-1)**(int(bit_string[j+1]))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,13)))))    
        if j>=n_site-2:                                    
            mag2=mag2+a2*obs*count/(zero)/(1)
        else:
            mag2=mag2
print('mag2=',mag2)

############################################  XX-energy
qc.h(qr[12])
qc.h(qr[13])
qc_total_xx=qc.compose(qc_meas)
qc.h(qr[12])
qc.h(qr[13])
    

job=execute(qc_total_xx, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_xx)
#print('xx-counts=', counts)
       
magx=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,13)))))*count 
    magx=a4*obs/(zero)
print('magx=', magx)

#################################
    
qc_total_zz=qc.compose(qc_meas)
  
job=execute(qc_total_zz, simulator, shots=n_shots)
result=job.result()
counts=result.get_counts(qc_total_zz)
       
magzz=0
obs=0
for bit_string, count in counts.items():
    for j in range(1):
        obs=obs+((-1)**((int(bit_string[0]))+(int(bit_string[1]))))*((reduce(mul,((1-int(bit_string[i])) for i in range(2,13)))))*count 
    magzz=a3*obs/(zero)
print('magzz=', magzz)

###################################
print('EE=', mag00+magzz+mag+mag2+magx)
print('E0=', a00+magzz+mag+mag2+magx)

######  tomography for E1,E2


zero= 9874609
mag00= 0.0
mag= -0.41123669222244646
mag2= -0.41123669222244646
magx= -0.037322923609937365
magzz= 0.011499982530953882
EE= -0.8482963255238765
E0= -1.8922063255238761
